# 실무예제 4-1
## 다음은 2015년 국내대학현황이다. 데이터큐브 집계를 이용하여 학제별/지역별/설립별로 재학생수 합계로 데이터를 축소시키시오.
### 데이터 파일 : ch4-1(국내대학현황).csv
### 원본 투플수 : 1,930개

In [27]:
# ch4-1.py
import pandas as pd
import numpy as np
import pymssql      # SQLServer DB 연동을 위한 pymssql 패키지 임포트

# 데이터로드 (ch4-1.csv : 데이터 원본 파일)
# encoding : 윈도우즈 환경에서의 한글 처리
# engine : python 3.6에서 한글이 포함된 파일이름 사용
rawData = pd.read_csv('jupyter/ch4-1(국내대학현황).csv', encoding='CP949', engine='python')

In [28]:
rawData

,학제,학교명,지역,설립,재적학생수,재학생수,휴학생수,총장및전임교원수
0,전문대학,한국철도대학,경기,국립,180.0,134.0,46.0,NaN
1,전문대학,강원도립대학,강원,공립,1658.0,900.0,758.0,31.0
2,전문대학,경남도립거창대학,경남,공립,1614.0,955.0,659.0,31.0
3,전문대학,경남도립남해대학,경남,공립,1639.0,846.0,793.0,25.0
4,전문대학,경북도립대학교,경북,공립,1389.0,930.0,459.0,32.0
...,...,...,...,...,...,...,...,...
1925,대학부설대학원,한국과학기술원 일반대학원,대전,특별법법인,6962.0,6648.0,314.0,120.0
1926,대학부설대학원,광주과학기술원 일반대학원,광주,특별법법인,1167.0,1144.0,23.0,131.0
1927,대학부설대학원,대구경북과학기술원 일반대학원,대구,특별법법인,321.0,314.0,7.0,51.0
1928,대학부설대학원,한국전통문화대학교 일반대학원,충남,특별법국립,35.0,34.0,1.0,NaN


In [29]:
# MSSQL DB 연결
# 접속정보(connection string) : ID/PASS@CONNECTION_ALIAS
# CONNECTION_ALIAS : Oracle TNSNAMES.ORA 파일에 있는 접속정보 별칭(ALIAS)
conn_mssql = pymssql.connect(server="localhost", database="prep1")

# DB 커서(Cursor) 선언
cur = conn_mssql.cursor()

# 사용할 MSSQL 소스 테이블명 지정
src_table = "d_base4_1"

# 데이터프레임(rawData)에 저장된 데이터를 MSSQL 테이블(d_base4_1)에 입력하기 위한 로직
# d_base4_1 테이블 존재하는지 체크하는 함수
def table_exists(name=None, con=None):
    sql = "select * from information_schema.tables where table_name='MYTABLE'".replace('MYTABLE', name.upper())
    df = pd.read_sql(sql, con)

    # 테이블이 존재하면 True, 그렇지 않으면 False 반환
    exists = True if len(df) > 0 else False
    return exists

# 테이블(d_base4_1) 생성 (테이블이 이미 존재한다면 TRUNCATE TABLE)
if table_exists(src_table, conn_mssql):
    cur.execute("TRUNCATE TABLE " + src_table)
else:
    cur.execute("create table " + src_table + " ( \
               학제 varchar(40), \
               학교명 varchar(100), \
               지역 varchar(10), \
               설립 varchar(20), \
               재적학생수 numeric(8), \
               재학생수 numeric(8), \
               휴학생수 numeric(8), \
               총장및전임교원수 numeric(8))")

# INSERT INTO ... VALUES (%s, %s, ...) 에서 바인드 변수값을 주기위해 tuple 구조 사용
def convertSequenceToList(list):
    dict = []
    for v in list :
        if pd.isnull(v):
            dict.append(-1) # list 요소 값이 NULL이면 -1로 assign
        else:
            dict.append(v)
    return tuple(dict)

# 데이터프레임에 저장된 데이터를 MSSQL 테이블로 입력(insert)
cols = [k for k in rawData.dtypes.index]
colnames = ','.join(cols)
colpos = ', '.join(['%s' for f in enumerate(cols)])
insert_sql = 'INSERT INTO %s (%s) VALUES (%s)' % (src_table, colnames, colpos)

# INSERT INTO ... VALUES (%s, %s, ...)의 바인드 변수 값을 저장하는 tuple 구조 생성
data = [convertSequenceToList(rec) for rec in rawData.values]

# 바인드 변수와 tuple 데이터구조를 활용하여 Bulk Insertion 구현
cur.executemany(insert_sql, data)

# 컬럼값이 -1이면, NULL값으로 업데이트
cur.execute("update " + src_table +
           " set 재적학생수 = case when 재적학생수 = -1 then NULL else 재적학생수 end \
             , 재학생수 = case when 재학생수 = -1 then NULL else 재학생수 end \
             , 휴학생수 = case when 휴학생수 = -1 then NULL else 휴학생수 end \
             , 총장및전임교원수 = case when 총장및전임교원수 = -1 then NULL else 총장및전임교원수 end \
           where 재적학생수 = -1 OR 재학생수 = -1 OR 휴학생수 = -1 OR 총장및전임교원수 = -1")

# csv 파일 데이터의 MSSQL 테이블 입력 완료 
conn_mssql.commit()

### 실무예제 2-2 / 3-2의 MSSQL 연동 설명 참조
### MSSQL Studio를 통해서 d_base4_1 테이블 생성 확인
### ‘’ 또는 ‘NULL’은 모두 string으로 인식되면서 cur.executemany() 함수에서 TypeError를 일으킴. 이를 해결하기 위해
#### csv 파일에서 숫자형식 속성값이 NULL이면, -1(도저히 있을 수 없는 속성값 중 하나)로 치환
#### 추후 SQL Update문을 이용하여 –1값은 NULL로 다시 업데이터하는 방법을 사용함

In [30]:
def view_exists(name=None, con=None):
    sql = "select * from information_schema.views where table_name='MYVIEW'".replace('MYVIEW', name.upper())
    df = pd.read_sql(sql, con)

    # 테이블이 존재하면 True, 그렇지 않으면 False 반환
    exists = True if len(df) > 0 else False
    return exists

# 뷰(v_base4_1) 생성 (뷰가 이미 존재한다면 DROP VIEW)
if view_exists('v_base4_1', conn_mssql):
    cur.execute("drop view v_base4_1") 

# cube(), grouping() 함수를 이용한 데이터큐브 뷰(view) 생성
cur.execute("""create view v_base4_1 
               as 
               select case when grouping(지역) = 1 then '총계' else 지역 end 지역 
                    , case when grouping(학제) = 1 then '총계' else 학제 end 학제 
                    , case when grouping(설립) = 1 then '총계' else 설립 end 설립 
                    , sum(재학생수) 재학생수 
               from """ + src_table + " group by cube(지역, 학제, 설립)")

### MSSQL의 cube() 함수를 이용하여 데이터큐브를 뷰(view)로 생성
#### grouping(A) : 속성 A가 집계된 결과이면 1을, 그렇지 않으면 0을 돌려주는 오라클 함수
#### 학제, 지역, 설립 등 3가지 속성에 대한 3차원 큐브로서 총 8가지의 집계 레벨이 도출됨
### 데이터큐브 뷰에 대하여 where 조건을 명시함으로서 8가지의 각기 다른 차원의 집계 데이터를 얻을 수 있음

In [31]:
# (1) 지역/학제/설립 별 재학생수
result_df = pd.read_sql("select 지역, 학제, 설립, 재학생수 \
                     from v_base4_1 \
                     where 지역 <> '총계' and 학제 <> '총계' and 설립 <> '총계'", con=conn_mssql)
print("(1) 지역/학제/설립 별 재학생수")
result_df  # 결과보기

(1) 지역/학제/설립 별 재학생수


,지역,학제,설립,재학생수
0,서울,대학교,공립,9516.0
1,서울,대학부설대학원,공립,2679.0
2,강원,전문대학,공립,900.0
3,경남,전문대학,공립,1801.0
4,경북,전문대학,공립,930.0
...,...,...,...,...
178,대전,대학교,특별법법인,3968.0
179,광주,대학부설대학원,특별법법인,1144.0
180,대구,대학부설대학원,특별법법인,314.0
181,대전,대학부설대학원,특별법법인,6648.0


In [32]:
# (2) 지역/학제 별 재학생수(설립 집계 레벨)
result_df = pd.read_sql("select 지역, 학제, 재학생수 \
                     from v_base4_1 \
                     where 지역 <> '총계' and 학제 <> '총계' and 설립 = '총계'", con=conn_mssql)
print("(2) 지역/학제 별 재학생수(설립 집계 레벨)")
result_df  # 결과보기

(2) 지역/학제 별 재학생수(설립 집계 레벨)


,지역,학제,재학생수
0,대전,각종대학(대학),10.0
1,서울,각종대학(대학),2652.0
2,충북,각종대학(대학),300.0
3,경기,각종대학(전문),9.0
4,강원,교육대학,1395.0
...,...,...,...
118,전남,전문대학,21176.0
119,전북,전문대학,19339.0
120,제주,전문대학,7585.0
121,충남,전문대학,16830.0


In [33]:
# (3) 학제/설립 별 재학생수(지역 집계 레벨)
result_df = pd.read_sql("select 학제, 설립, 재학생수 \
                     from v_base4_1 \
                     where 지역 = '총계' and 학제 <> '총계' and 설립 <> '총계'", con=conn_mssql)
print("(3) 학제/설립 별 재학생수(지역 집계 레벨)")
result_df  # 결과보기

(3) 학제/설립 별 재학생수(지역 집계 레벨)


,학제,설립,재학생수
0,대학교,공립,9516.0
1,대학부설대학원,공립,2679.0
2,전문대학,공립,7243.0
3,교육대학,국립,15525.0
4,대학교,국립,299708.0
5,대학부설대학원,국립,70804.0
6,방송통신대학,국립,134469.0
7,산업대학,국립,10236.0
8,전문대학,국립,1853.0
9,대학교,국립대법인,32550.0


In [34]:
# (4) 지역/설립 별 재학생수(학제 집계 레벨)
result_df = pd.read_sql("select 지역, 설립, 재학생수 \
                     from v_base4_1 \
                     where 지역 <> '총계' and 학제 = '총계' and 설립 <> '총계'", con=conn_mssql)
print("(4) 지역/설립 별 재학생수(학제 집계 레벨)")
result_df  # 결과보기

(4) 지역/설립 별 재학생수(학제 집계 레벨)


,지역,설립,재학생수
0,강원,공립,900.0
1,강원,국립,35636.0
2,강원,국립대법인,16.0
3,강원,사립,61081.0
4,경기,국립,8479.0
5,경기,사립,348163.0
6,경기,특별법법인,321.0
7,경남,공립,1801.0
8,경남,국립,37082.0
9,경남,사립,59070.0


In [35]:
# (5) 지역 별 재학생수(학제/설립 집계 레벨)
result_df = pd.read_sql("select 지역, 재학생수 \
                     from v_base4_1 \
                     where 지역 <> '총계' and 학제 = '총계' and 설립 = '총계'", con=conn_mssql)
print("(5) 지역 별 재학생수(학제/설립 집계 레벨)")
result_df # 결과보기

(5) 지역 별 재학생수(학제/설립 집계 레벨)


,지역,재학생수
0,강원,97633.0
1,경기,356963.0
2,경남,97953.0
3,경북,172855.0
4,광주,96208.0
5,대구,102222.0
6,대전,131027.0
7,부산,206554.0
8,서울,768766.0
9,세종,18631.0


In [36]:
# (6) 학제 별 재학생수(지역/설립 집계 레벨)
result_df = pd.read_sql("select 학제, 재학생수 \
                     from v_base4_1 \
                     where 지역 = '총계' and 학제 <> '총계' and 설립 = '총계'", con=conn_mssql)
print("(6) 학제 별 재학생수(지역/설립 집계 레벨)")
result_df  # 결과보기

(6) 학제 별 재학생수(지역/설립 집계 레벨)


,학제,재학생수
0,각종대학(대학),2962.0
1,각종대학(전문),9.0
2,교육대학,15525.0
3,기능대학,17981.0
4,기술대학,112.0
5,대학교,1520850.0
6,대학부설대학원,280794.0
7,대학원대학,8081.0
8,방송통신대학,134469.0
9,사내대학(대학),257.0


In [37]:
# (7) 설립 별 재학생수(지역/학제 집계 레벨)
result_df = pd.read_sql("select 설립, 재학생수 \
                     from v_base4_1 \
                     where 지역 = '총계' and 학제 = '총계' and 설립 <> '총계'", con=conn_mssql)
print("(7) 설립 별 재학생수(지역/학제 집계 레벨)")
result_df  # 결과보기

(7) 설립 별 재학생수(지역/학제 집계 레벨)


,설립,재학생수
0,공립,19438.0
1,국립,532595.0
2,국립대법인,46851.0
3,기타,1076.0
4,사립,1974947.0
5,특별법국립,3325.0
6,특별법법인,13320.0


In [38]:
# (8) 총 재학생수 (지역/학제/설립 집계 레벨)
result_df = pd.read_sql("select 재학생수 \
                     from v_base4_1 \
                     where 지역 = '총계' and 학제 = '총계' and 설립 = '총계'", con=conn_mssql)
print("(8) 총 재학생수 (지역/학제/설립 집계 레벨)")
result_df  # 결과보기

(8) 총 재학생수 (지역/학제/설립 집계 레벨)


,재학생수
0,2591552.0
